In [1]:
#inspiration for bokeh viz
#https://www.geeksforgeeks.org/python-bokeh-plotting-a-scatter-plot-on-a-graph/
#https://github.com/WillKoehrsen/Bokeh-Python-Visualization/blob/master/interactive/exploration/delay_histogram.py

In [2]:
#World Health Organization API
#https://www.who.int/data/gho/info/gho-odata-api

#World Bank API

    

In [3]:
#Import the basics
import pandas as pd
import numpy as np


import requests as r
import json
import time

#import country converter
#this will be a time saver
#https://nbviewer.jupyter.org/github/konstantinstadler/country_converter/blob/master/doc/country_converter_examples.ipynb



import sys
!{sys.executable} -m pip install country_converter

In [4]:
import country_converter as coco
converter = coco.CountryConverter()




In [5]:
converter.data

,APEC,BASIC,BRIC,CIS,Cecilia2050,EEA,EU,EU12,EU15,EU25,...,UN,UNcode,UNmember,UNregion,WIOD,continent,name_official,name_short,obsolete,regex
0,NaN,NaN,NaN,NaN,RoW,NaN,NaN,NaN,NaN,NaN,...,1946.0,4.0,1946.0,Southern Asia,RoW,Asia,Islamic Republic of Afghanistan,Afghanistan,NaN,afghan
1,NaN,NaN,NaN,NaN,RoW,NaN,NaN,NaN,NaN,NaN,...,NaN,248.0,NaN,Northern Europe,RoW,Europe,Åland Islands,Aland Islands,NaN,\b(a|å)land
2,NaN,NaN,NaN,NaN,RoW,NaN,NaN,NaN,NaN,NaN,...,1955.0,8.0,1955.0,Southern Europe,RoW,Europe,Republic of Albania,Albania,NaN,albania
3,NaN,NaN,NaN,NaN,RoW,NaN,NaN,NaN,NaN,NaN,...,1962.0,12.0,1962.0,Northern Africa,RoW,Africa,People's Democratic Republic of Algeria,Algeria,NaN,algeria
4,NaN,NaN,NaN,NaN,RoW,NaN,NaN,NaN,NaN,NaN,...,NaN,16.0,NaN,Polynesia,RoW,Oceania,American Samoa,American Samoa,NaN,^(?=.*americ).*samoa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,NaN,NaN,NaN,NaN,RoW,NaN,NaN,NaN,NaN,NaN,...,NaN,876.0,NaN,Polynesia,RoW,Oceania,Wallis and Futuna Islands,Wallis and Futuna Islands,NaN,futuna|wallis
246,NaN,NaN,NaN,NaN,RoW,NaN,NaN,NaN,NaN,NaN,...,NaN,732.0,NaN,Northern Africa,RoW,Africa,Western Sahara,Western Sahara,NaN,\bw.*sahara
247,NaN,NaN,NaN,NaN,RoW,NaN,NaN,NaN,NaN,NaN,...,1947.0,887.0,1947.0,Western Asia,RoW,Asia,Republic of Yemen,Yemen,NaN,yemen
248,NaN,NaN,NaN,NaN,RoW,NaN,NaN,NaN,NaN,NaN,...,1964.0,894.0,1964.0,Eastern Africa,RoW,Africa,Republic of Zambia,Zambia,NaN,zambia|northern.?rhodesia


In [6]:
#Import the visualization tools from bokeh

from bokeh.io import show, curdoc, output_notebook
from bokeh.plotting import figure, output_file, show
#from bokeh.plotting import figure, output_file, show 

from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs, RadioButtonGroup, MultiChoice
from bokeh.layouts import column, row, WidgetBox

#palettes
from bokeh.palettes import Magma256

#init bokeh
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler

In [7]:

#who URL
url = 'https://ghoapi.azureedge.net/api/SA_0000001400?$filter=date(TimeDimensionBegin) ge 2001-01-01 and date(TimeDimensionBegin) lt 2020-01-01'


response = r.request('GET', url )
pull = json.loads(response.text)

#print(pull)

In [8]:
#test = pull['value']
#print(test)

In [9]:
whoData = []
colswdf = ['ISOAlphaCountry','year','alcType', 'NumValue', 'lowBound','hiBound']


for line in pull['value']:
    ISOAlphaCountry = line['SpatialDim']
    year = line['TimeDim']
    alcType = line['Dim1']
    NumValue = line['NumericValue']
    lowBound = line['Low']
    hiBound = line['High']
    
    whoData.append([ISOAlphaCountry,year,alcType, NumValue, lowBound,hiBound])

In [10]:
whoData[0]

['AFG', 2005, 'SA_BEER', 0.00934, 0.00572, 0.01309]

In [11]:
whoDF = pd.DataFrame(whoData, columns=colswdf)

whoDF.tail()

,ISOAlphaCountry,year,alcType,NumValue,lowBound,hiBound
17720,ZWE,2019,SA_BEER,1.20,0.72478,1.62654
17721,ZWE,2019,SA_WINE,0.05,0.02939,0.07011
17722,ZWE,2019,SA_SPIRITS,0.39,0.23672,0.54334
17723,ZWE,2019,SA_OTHER_ALCOHOL,1.47,0.85331,2.05113
17724,ZWE,2019,SA_TOTAL,3.11,2.34685,3.86315


In [10]:
#pivot to make it wide with countryCode and year as the index

In [12]:
#get country codes and names

whoWide = whoDF.pivot_table(index=["ISOAlphaCountry","year"], columns="alcType", values="NumValue").reset_index()

whoWide.tail()

alcType,ISOAlphaCountry,year,SA_BEER,SA_OTHER_ALCOHOL,SA_SPIRITS,SA_TOTAL,SA_WINE
3540,ZWE,2015,1.93,1.49,0.36,3.84,0.06
3541,ZWE,2016,1.93,1.44,0.35,3.76,0.05
3542,ZWE,2017,1.93,1.45,0.39,3.82,0.05
3543,ZWE,2018,1.20,1.47,0.39,3.10,0.04
3544,ZWE,2019,1.20,1.47,0.39,3.11,0.05


In [13]:
#use country converter to get name and continent
whoWide['CountryName'] = converter.convert(names = whoWide['ISOAlphaCountry'], src = 'ISO3', to = 'name_short')
whoWide['UNregion'] = converter.convert(names = whoWide['ISOAlphaCountry'], src = 'ISO3', to = 'UNregion')

In [14]:
whoWide.tail()

alcType,ISOAlphaCountry,year,SA_BEER,SA_OTHER_ALCOHOL,SA_SPIRITS,SA_TOTAL,SA_WINE,CountryName,UNregion
3540,ZWE,2015,1.93,1.49,0.36,3.84,0.06,Zimbabwe,Eastern Africa
3541,ZWE,2016,1.93,1.44,0.35,3.76,0.05,Zimbabwe,Eastern Africa
3542,ZWE,2017,1.93,1.45,0.39,3.82,0.05,Zimbabwe,Eastern Africa
3543,ZWE,2018,1.20,1.47,0.39,3.10,0.04,Zimbabwe,Eastern Africa
3544,ZWE,2019,1.20,1.47,0.39,3.11,0.05,Zimbabwe,Eastern Africa


Rename some of the dimensions to nicer names

In [15]:
whoWide.rename({'SA_TOTAL': 'Total_LAE_PCC',
                'SA_BEER': 'Beer_LAE_PCC',
                'SA_SPIRITS': 'Spirits_LAE_PCC',
                'SA_WINE': 'Wine_LAE_PCC',
                'SA_OTHER_ALCOHOL': 'Other_LAE_PCC'}, axis=1, inplace=True)

In [16]:
whoWide['Formal_LAE_PCC'] = whoWide['Beer_LAE_PCC'] + whoWide['Wine_LAE_PCC'] + whoWide['Spirits_LAE_PCC'] 

In [17]:
whoWide.head()

alcType,ISOAlphaCountry,year,Beer_LAE_PCC,Other_LAE_PCC,Spirits_LAE_PCC,Total_LAE_PCC,Wine_LAE_PCC,CountryName,UNregion,Formal_LAE_PCC
0,AFG,2005,0.00934,0.0,0.00605,0.01623,0.00084,Afghanistan,Southern Asia,0.01623
1,AFG,2006,0.01147,0.0,0.00685,0.02618,0.00786,Afghanistan,Southern Asia,0.02618
2,AFG,2007,0.00713,0.0,0.00202,0.01801,0.00887,Afghanistan,Southern Asia,0.01802
3,AFG,2008,0.00857,0.0,0.01809,0.02756,0.00089,Afghanistan,Southern Asia,0.02755
4,AFG,2009,0.00972,0.0,0.00000,0.00972,0.00000,Afghanistan,Southern Asia,0.00972


In [18]:
#load world bank macro data
#http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL

#Households and NPISHs Final consumption expenditure, PPP (constant 2017 international $)
#NE.CON.PRVT.PP.KD

#population
#Population ages 15-64, total (
#SP.POP.1564.TO

#Population, total ()
#SP.POP.TOTL

wburl = 'http://api.worldbank.org/v2/country/all/indicator/NE.CON.PRVT.PP.KD;SP.POP.TOTL;SP.POP.1564.TO?source=2&date=2000:2020&format=json&per_page=20000'

wbresponse = r.request('GET', wburl )
wbpull = json.loads(wbresponse.text)

#print(wbpull)

       



In [19]:
meta = wbpull[0]
pages = meta['pages']
print(pages)

1


In [20]:
#WB like to give data in pages
#as an exercise we will use this and run through the pages and extract as necessary

wbjson = wbpull[1]
#print(wbData[1])

wbData = []
wbCols = ['indicatorID','indicatorName','ISOAlphaCountry','year','value']

for line in wbjson:
    indicatorID = line['indicator']['id']
    indicatorName = line['indicator']['value']
    ISOAlphaCountry = line['countryiso3code']
    year = line['date']
    value = line['value']
    
    wbData.append([indicatorID,indicatorName,ISOAlphaCountry,year,value])
    
    
wbDF = pd.DataFrame(wbData, columns=wbCols)

wbDF.tail() 


,indicatorID,indicatorName,ISOAlphaCountry,year,value
16627,SP.POP.1564.TO,"Population ages 15-64, total",ZWE,2004,6672931.0
16628,SP.POP.1564.TO,"Population ages 15-64, total",ZWE,2003,6638709.0
16629,SP.POP.1564.TO,"Population ages 15-64, total",ZWE,2002,6603510.0
16630,SP.POP.1564.TO,"Population ages 15-64, total",ZWE,2001,6567476.0
16631,SP.POP.1564.TO,"Population ages 15-64, total",ZWE,2000,6528385.0


In [21]:
wbWide = wbDF.pivot_table(index=["ISOAlphaCountry","year"], columns="indicatorID", values="value").reset_index()

wbWide.tail()

indicatorID,ISOAlphaCountry,year,NE.CON.PRVT.PP.KD,SP.POP.1564.TO,SP.POP.TOTL
5187,ZWE,2015,3.972888e+10,7543265.0,13814629.0
5188,ZWE,2016,3.556373e+10,7642793.0,14030390.0
5189,ZWE,2017,3.606260e+10,7759872.0,14236745.0
5190,ZWE,2018,3.365083e+10,7892282.0,14439018.0
5191,ZWE,2019,NaN,8034689.0,14645468.0


In [22]:
#rename some columns

wbWide.rename({'NE.CON.PRVT.PP.KD': 'HCE_Total',
                'SP.POP.1564.TO': 'Pop_15_64',
                'SP.POP.TOTL': 'Pop_Total'}, axis=1, inplace=True)

In [18]:
#We have three data frames now. One is like a country dimension table, then two value tables
# we want to create one df by with countryCode and year as the index / rows, and the indicators going wide
#use WHO as the main data and left merge the WB data onto it

In [23]:
whoWide.dtypes



alcType
ISOAlphaCountry     object
year                 int64
Beer_LAE_PCC       float64
Other_LAE_PCC      float64
Spirits_LAE_PCC    float64
Total_LAE_PCC      float64
Wine_LAE_PCC       float64
CountryName         object
UNregion            object
Formal_LAE_PCC     float64
dtype: object

In [24]:
wbWide.dtypes

indicatorID
ISOAlphaCountry     object
year                object
HCE_Total          float64
Pop_15_64          float64
Pop_Total          float64
dtype: object

In [49]:
#set our merging types to strings for easier merging
whoWide['year'] = whoWide['year'].astype(str)
wbWide['year'] = wbWide['year'].astype(str)

In [81]:
joined = pd.merge(whoWide, wbWide, how='left', on=['ISOAlphaCountry', 'year'])

In [64]:
joined.tail()

,ISOAlphaCountry,year,Beer_LAE_PCC,Other_LAE_PCC,Spirits_LAE_PCC,Total_LAE_PCC,Wine_LAE_PCC,CountryName,UNregion,Formal_LAE_PCC,HCE_Total,Pop_15_64,Pop_Total
3540,ZWE,2015,1.93,1.49,0.36,3.84,0.06,Zimbabwe,Eastern Africa,2.35,3.972888e+10,7543265.0,13814629.0
3541,ZWE,2016,1.93,1.44,0.35,3.76,0.05,Zimbabwe,Eastern Africa,2.33,3.556373e+10,7642793.0,14030390.0
3542,ZWE,2017,1.93,1.45,0.39,3.82,0.05,Zimbabwe,Eastern Africa,2.37,3.606260e+10,7759872.0,14236745.0
3543,ZWE,2018,1.20,1.47,0.39,3.10,0.04,Zimbabwe,Eastern Africa,1.63,3.365083e+10,7892282.0,14439018.0
3544,ZWE,2019,1.20,1.47,0.39,3.11,0.05,Zimbabwe,Eastern Africa,1.64,NaN,8034689.0,14645468.0


do some aggregating by continent to create some "new" countries for selecting



In [82]:
joined['Total_LAE'] = joined['Total_LAE_PCC'] * joined['Pop_15_64']
joined['Beer_LAE'] = joined['Beer_LAE_PCC'] * joined['Pop_15_64']
joined['Spirits_LAE'] = joined['Spirits_LAE_PCC'] * joined['Pop_15_64']
joined['Wine_LAE'] = joined['Wine_LAE_PCC'] * joined['Pop_15_64']
joined['Other_LAE'] = joined['Other_LAE_PCC'] * joined['Pop_15_64']
joined['Formal_LAE'] = joined['Formal_LAE_PCC'] * joined['Pop_15_64']

In [83]:
conpvt_values = ['Total_LAE','Beer_LAE','Spirits_LAE','Wine_LAE','Other_LAE','Formal_LAE','HCE_Total','Pop_15_64','Pop_Total']

In [84]:
continent_annual = pd.pivot_table(joined, values=conpvt_values, 
                                  index=['UNregion', 'year'],
                                  aggfunc=np.sum).reset_index()

In [85]:
continent_annual

,UNregion,year,Beer_LAE,Formal_LAE,HCE_Total,Other_LAE,Pop_15_64,Pop_Total,Spirits_LAE,Total_LAE,Wine_LAE
0,Australia and New Zealand,2001,7.299768e+07,1.440104e+08,4.408683e+11,1.968920e+06,15541208.0,23293500.0,2.681202e+07,1.459793e+08,4.420068e+07
1,Australia and New Zealand,2002,7.650149e+07,1.512651e+08,4.556443e+11,2.032474e+06,15776253.0,23599900.0,2.799138e+07,1.532975e+08,4.677220e+07
2,Australia and New Zealand,2003,7.735234e+07,1.549080e+08,4.753729e+11,2.023581e+06,16019611.0,23922600.0,3.021376e+07,1.569315e+08,4.734186e+07
3,Australia and New Zealand,2004,7.296670e+07,1.432177e+08,4.995366e+11,1.477102e+07,16241092.0,24214900.0,2.016795e+07,1.579888e+08,5.008310e+07
4,Australia and New Zealand,2005,7.467372e+07,1.517283e+08,5.222881e+11,1.526252e+07,16473162.0,24528700.0,2.037520e+07,1.669908e+08,5.667939e+07
...,...,...,...,...,...,...,...,...,...,...,...
413,Western Europe,2015,5.380645e+08,1.369715e+09,4.588072e+12,1.136173e+07,125462154.0,193943701.0,2.590005e+08,1.381021e+09,5.726499e+08
414,Western Europe,2016,5.347492e+08,1.353820e+09,4.679130e+12,1.149843e+07,125792804.0,195126529.0,2.585648e+08,1.365375e+09,5.605064e+08
415,Western Europe,2017,5.323541e+08,1.345550e+09,4.753212e+12,1.107108e+07,125842936.0,195873577.0,2.578418e+08,1.356329e+09,5.553536e+08
416,Western Europe,2018,5.384335e+08,1.327710e+09,4.817853e+12,1.001096e+07,125804996.0,196493172.0,2.547971e+08,1.337721e+09,5.344795e+08


In [86]:
continent_annual['Total_LAE_PCC'] = continent_annual['Total_LAE'] / continent_annual['Pop_15_64']
continent_annual['Beer_LAE_PCC'] = continent_annual['Beer_LAE'] / continent_annual['Pop_15_64']
continent_annual['Spirits_LAE_PCC'] = continent_annual['Spirits_LAE'] / continent_annual['Pop_15_64']
continent_annual['Wine_LAE_PCC'] = continent_annual['Wine_LAE'] / continent_annual['Pop_15_64']
continent_annual['Other_LAE_PCC'] = continent_annual['Other_LAE'] / continent_annual['Pop_15_64']
continent_annual['Formal_LAE_PCC'] = continent_annual['Formal_LAE'] / continent_annual['Pop_15_64']



add country name variable continent to country name so we can use it in the multiselect tool

In [87]:
#hack - add an underscore in front of these so they are at the top of the list

continent_annual['CountryName'] = 'AA' + continent_annual['UNregion']



continent_annual.head()



,UNregion,year,Beer_LAE,Formal_LAE,HCE_Total,Other_LAE,Pop_15_64,Pop_Total,Spirits_LAE,Total_LAE,Wine_LAE,Total_LAE_PCC,Beer_LAE_PCC,Spirits_LAE_PCC,Wine_LAE_PCC,Other_LAE_PCC,Formal_LAE_PCC,CountryName
0,Australia and New Zealand,2001,7.299768e+07,1.440104e+08,4.408683e+11,1.968920e+06,15541208.0,23293500.0,2.681202e+07,1.459793e+08,4.420068e+07,9.393046,4.697040,1.725221,2.844095,0.126690,9.266356,AAAustralia and New Zealand
1,Australia and New Zealand,2002,7.650149e+07,1.512651e+08,4.556443e+11,2.032474e+06,15776253.0,23599900.0,2.799138e+07,1.532975e+08,4.677220e+07,9.716977,4.849155,1.774273,2.964722,0.128831,9.588149,AAAustralia and New Zealand
2,Australia and New Zealand,2003,7.735234e+07,1.549080e+08,4.753729e+11,2.023581e+06,16019611.0,23922600.0,3.021376e+07,1.569315e+08,4.734186e+07,9.796214,4.828603,1.886048,2.955244,0.126319,9.669895,AAAustralia and New Zealand
3,Australia and New Zealand,2004,7.296670e+07,1.432177e+08,4.995366e+11,1.477102e+07,16241092.0,24214900.0,2.016795e+07,1.579888e+08,5.008310e+07,9.727718,4.492721,1.241785,3.083727,0.909484,8.818234,AAAustralia and New Zealand
4,Australia and New Zealand,2005,7.467372e+07,1.517283e+08,5.222881e+11,1.526252e+07,16473162.0,24528700.0,2.037520e+07,1.669908e+08,5.667939e+07,10.137143,4.533053,1.236872,3.440711,0.926508,9.210636,AAAustralia and New Zealand


In [88]:
joined = pd.concat([joined, continent_annual])

In [89]:
joined.tail()

,ISOAlphaCountry,year,Beer_LAE_PCC,Other_LAE_PCC,Spirits_LAE_PCC,Total_LAE_PCC,Wine_LAE_PCC,CountryName,UNregion,Formal_LAE_PCC,HCE_Total,Pop_15_64,Pop_Total,Total_LAE,Beer_LAE,Spirits_LAE,Wine_LAE,Other_LAE,Formal_LAE
413,NaN,2015,4.288660,0.090559,2.064372,11.007471,4.564324,AAWestern Europe,Western Europe,10.917356,4.588072e+12,125462154.0,193943701.0,1.381021e+09,5.380645e+08,2.590005e+08,5.726499e+08,11361734.68,1.369715e+09
414,NaN,2016,4.251032,0.091408,2.055481,10.854157,4.455790,AAWestern Europe,Western Europe,10.762303,4.679130e+12,125792804.0,195126529.0,1.365375e+09,5.347492e+08,2.585648e+08,5.605064e+08,11498428.70,1.353820e+09
415,NaN,2017,4.230306,0.087975,2.048917,10.777953,4.413069,AAWestern Europe,Western Europe,10.692293,4.753212e+12,125842936.0,195873577.0,1.356329e+09,5.323541e+08,2.578418e+08,5.553536e+08,11071083.15,1.345550e+09
416,NaN,2018,4.279905,0.079575,2.025333,10.633290,4.248476,AAWestern Europe,Western Europe,10.553715,4.817853e+12,125804996.0,196493172.0,1.337721e+09,5.384335e+08,2.547971e+08,5.344795e+08,10010963.71,1.327710e+09
417,NaN,2019,4.269631,0.082307,1.960829,10.572819,4.259289,AAWestern Europe,Western Europe,10.489749,4.888821e+12,125721226.0,197071595.0,1.329228e+09,5.367832e+08,2.465179e+08,5.354830e+08,10347798.12,1.318784e+09


In [90]:
joined['HCE_PCC'] = joined['HCE_Total'] / joined['Pop_Total'] 

In [91]:
joined.tail()

,ISOAlphaCountry,year,Beer_LAE_PCC,Other_LAE_PCC,Spirits_LAE_PCC,Total_LAE_PCC,Wine_LAE_PCC,CountryName,UNregion,Formal_LAE_PCC,HCE_Total,Pop_15_64,Pop_Total,Total_LAE,Beer_LAE,Spirits_LAE,Wine_LAE,Other_LAE,Formal_LAE,HCE_PCC
413,NaN,2015,4.288660,0.090559,2.064372,11.007471,4.564324,AAWestern Europe,Western Europe,10.917356,4.588072e+12,125462154.0,193943701.0,1.381021e+09,5.380645e+08,2.590005e+08,5.726499e+08,11361734.68,1.369715e+09,23656.718983
414,NaN,2016,4.251032,0.091408,2.055481,10.854157,4.455790,AAWestern Europe,Western Europe,10.762303,4.679130e+12,125792804.0,195126529.0,1.365375e+09,5.347492e+08,2.585648e+08,5.605064e+08,11498428.70,1.353820e+09,23979.980634
415,NaN,2017,4.230306,0.087975,2.048917,10.777953,4.413069,AAWestern Europe,Western Europe,10.692293,4.753212e+12,125842936.0,195873577.0,1.356329e+09,5.323541e+08,2.578418e+08,5.553536e+08,11071083.15,1.345550e+09,24266.731986
416,NaN,2018,4.279905,0.079575,2.025333,10.633290,4.248476,AAWestern Europe,Western Europe,10.553715,4.817853e+12,125804996.0,196493172.0,1.337721e+09,5.384335e+08,2.547971e+08,5.344795e+08,10010963.71,1.327710e+09,24519.185709
417,NaN,2019,4.269631,0.082307,1.960829,10.572819,4.259289,AAWestern Europe,Western Europe,10.489749,4.888821e+12,125721226.0,197071595.0,1.329228e+09,5.367832e+08,2.465179e+08,5.354830e+08,10347798.12,1.318784e+09,24807.333310


In [92]:
#get list of countries for the checkbox
my_countries = list(joined['CountryName'].unique())

my_countries = sorted(my_countries)

print(my_countries)

['AAAustralia and New Zealand', 'AACaribbean', 'AACentral America', 'AACentral Asia', 'AAEastern Africa', 'AAEastern Asia', 'AAEastern Europe', 'AAMelanesia', 'AAMicronesia', 'AAMiddle Africa', 'AANorthern Africa', 'AANorthern America', 'AANorthern Europe', 'AAPolynesia', 'AASouth America', 'AASouth-Eastern Asia', 'AASouthern Africa', 'AASouthern Asia', 'AASouthern Europe', 'AAWestern Africa', 'AAWestern Asia', 'AAWestern Europe', 'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo Republic', 'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cu

In [93]:
#get dimensions for chart
my_dimensions = ['Total_LAE_PCC','Beer_LAE_PCC','Spirits_LAE_PCC','Wine_LAE_PCC', 'Formal_LAE_PCC','Other_LAE_PCC']

my_dimensions

['Total_LAE_PCC',
 'Beer_LAE_PCC',
 'Spirits_LAE_PCC',
 'Wine_LAE_PCC',
 'Formal_LAE_PCC',
 'Other_LAE_PCC']

In [94]:
#get years for checkbox

my_years = list(joined['year'].unique()) 

my_years.sort(reverse=True)

my_years

['2019',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013',
 '2012',
 '2011',
 '2010',
 '2009',
 '2008',
 '2007',
 '2006',
 '2005',
 '2004',
 '2003',
 '2002',
 '2001']

In [95]:
#file to save model
output_file('s_1.html')
output_notebook()


Loading BokehJS ...

In [96]:
#update data based on chart settings
#now, just based on list of countries from check box

def make_dataset(country_list, dim_list, year_list):
    
    #initialize dataframe to send to back as ColumnDataSource
    
    thisSet = pd.DataFrame(columns=['country', 'year', 'color', 
                                       'x', 'y', 'x_dim', 'y_dim'])
    
    #subset by countries first
    subset = joined[joined['CountryName'].isin(country_list)]    
    subset = subset[subset['year'].isin(year_list)]
    
    #print(subset)
    
    thisSet['country']=subset['CountryName']
    thisSet['year']=subset['year']
    thisSet['color']=subset['color']
    thisSet['x']=subset['HCE_PCC']
    thisSet['y']=subset[dim_list]
    thisSet=thisSet.assign(x_dim = "HCE Per Capita")
    thisSet=thisSet.assign(y_dim = str(dim_list) + " per Capita")
    
    
    return ColumnDataSource(thisSet)


In [97]:
#default styling for plot
#we can make our template here and easier to implement

def style(p):
    # Title 
    p.title.align = 'center'
    p.title.text_font_size = '20pt'
    p.title.text_font = 'serif'

    # Axis titles
    p.xaxis.axis_label_text_font_size = '14pt'
    p.xaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = '14pt'
    p.yaxis.axis_label_text_font_style = 'bold'

    # Tick labels
    p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.major_label_text_font_size = '12pt'

    return p




In [98]:
# standard function to make the scatter plot

def make_scatter(cds, dimension):

 
    dim = dimension
    p = figure(title='WHO ' + dim + ' vs HCE Per capita')
    p.xaxis.axis_label ='HCE per Capita ($PPP, 2017)'
    p.yaxis.axis_label = dim +' per Capita'
    size = 10
    
    p.scatter('x','y', source = cds, color='color', size=size, 
              fill_alpha=0.6, hover_fill_color='color', hover_fill_alpha=1.0 )
    
    #create and add hover tool to chart
    hover = HoverTool(tooltips=[('Country', '@country'), 
                                ('y_value', '@y'),
                                ('x_value', '@x'),
                                ('Year', '@year')],
                      mode='mouse')
    
    
    
    
    #create and add hover tool to chart
    #hover = HoverTool(tooltips=[('Country', '@WHO_Country'), 
    #                            ('PCC', '@Beer'),
    #                            ('HCE per cap', '@hce_cap'),
    #                            ('Year', '@WHO_Year')],
    #                  mode='mouse')
    
    p.add_tools(hover)
    
    #apply standard styling
    p = style(p)
    
    return p
    
    

In [99]:
#define the callback function to enable interactive updating with checkbox and sliders and such
#xxxxx
#need to get year selection in here as well

def update(attr, old, new):
    #since we are using check boxes, we are sending new data to the form
    
    #get selected countries
    countries_to_plot = country_selection.value
    new_years = [year_selection.labels[i] for i in year_selection.active]
    dim = dim_selection.labels[dim_selection.active]
    p.yaxis.axis_label = dim + " per capita"
    p.title.text ='WHO ' + dim + ' vs HCE Per capita'
    new_src = make_dataset(countries_to_plot, dim, new_years)
    
    src.data.update(new_src.data)


#create the object and name the axes

graph = figure(title='WHO SCATTER TOTAL ALC')
graph.xaxis.axis_label ='Income per Capita'
graph.yaxis.axis_label ='Alc (beer) per Capita'

size = 10

In [100]:
#set each country to a new color iterating through the palette
#currently set all to same color
def set_color(i):
 
    return Magma256[1]

#would need a country number for this.
#joined['color'] = joined.apply(lambda x: set_color(x['countryNum']), axis=1)

joined['color'] = joined.apply(lambda x: set_color(x['CountryName']), axis=1)

In [101]:
#initialize checkbox control
#geography
country_selection = MultiChoice(options=my_countries, value = ['Colombia'])
country_selection.on_change('value', update)

#year
year_selection = CheckboxGroup(labels=my_years, active = [1,2,3,4,5,6,7,8])
year_selection.on_change('active', update)


#dimension
dim_selection = RadioButtonGroup(labels=my_dimensions, active = 2)
dim_selection.on_change('active', update)

In [102]:
#dim_selection.active
#year_selection.labels[year_selection.active]

In [103]:
#find initially selected countries
#initial_countries = [country_selection.labels[i] for i in country_selection.active]
initial_countries = country_selection.value

initial_dim = dim_selection.labels[dim_selection.active]

initial_year = [year_selection.labels[i] for i in year_selection.active]

print(initial_countries)

['Colombia']


In [104]:
#create initial data set and create plot

src = make_dataset(initial_countries, initial_dim, initial_year)


p = make_scatter(src, initial_dim)

#print(src.data)

In [111]:
#put controls into a single element

#controls = column(dim_selection,year_selection,country_selection)
top = row(dim_selection)

bottom = row(country_selection,p,year_selection)


In [112]:
#create chart layout
#puts the controls on the left (or first) and plot on the right
#layout = row(controls, p)

layout = column(top,bottom) 



In [113]:
def modify_doc(doc):
    doc.add_root(row(layout, width=800))
        
    doc.title = "Checkboxes"



handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app)